# VacationPy - Sarah Martin

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
file = '../WeatherPy/Output/weather_data.csv'
city_df = pd.read_csv(file)
city_df.dropna(inplace=True)
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mae Hong Son,19.30,97.97,73.83,88,98,1.05,TH,1591734859
1,1,Témiscaming,46.72,-79.10,77.00,53,75,8.05,CA,1591734859
2,2,Ushuaia,-54.80,-68.30,41.00,87,75,2.75,AR,1591735199
3,3,Barrow,71.29,-156.79,30.20,80,90,5.82,US,1591735079
4,4,San Vicente,-26.62,-54.13,66.20,100,85,1.99,AR,1591734859


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Store longitude and latitude
locations=city_df[["Lat","Lng"]]

# Grab humidity values
humidity=city_df["Humidity"].astype(float)

# Plot heatmap
fig=gmaps.figure()

# Create heat layer based off humidity
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=max(city_df["Humidity"]),point_radius=4)

# Add the layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Creater data frame with ideal weather conditions.
ideal_df = city_df.loc[(city_df["Max Temp"] > 60) & (city_df["Max Temp"] < 75)]
ideal_df = ideal_df.loc[ideal_df["Wind Speed"] < 10]
ideal_df = ideal_df.loc[ideal_df["Cloudiness"] == 0]

ideal_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,Saurimo,-9.66,20.39,63.48,42,0,4.68,AO,1591734862
71,71,Carnarvon,-24.87,113.63,68.58,44,0,6.62,AU,1591735201
81,81,Crotone,39.09,17.12,68.22,70,0,8.23,IT,1591734867
90,90,Phalaborwa,-23.94,31.14,60.80,72,0,1.12,ZA,1591734868
150,150,Causapscal,48.35,-67.23,60.21,32,0,7.76,CA,1591734874


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
# Change the dataframe name and add a "Hotel Name" column
hotel_df=ideal_df
hotel_df["Hotel Name"]=""

# Establish Google API parameters and URL
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Begin looping through rows
for index, row in hotel_df.iterrows():
    # Grab the locations
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = (f"{lat},{lng}")
    
    # Make request
    print(f"Retreiving results for index {index}: {row['City']}.")
    response=requests.get(base_url,params=params).json()
    
    # Store results of the search
    results=response["results"]
    
    try:
        print(f"The closest hotel is {results[0]['Name']}.")
        hotel_df.loc[index,"Hotel Name"]=results[0]['Name']
        
    except (KeyError, IndexError):
        print("Missing field/result...skipping")
        
    print("-------------------")
        

Retreiving results for index 32: Saurimo.
Missing field/result...skipping
-------------------
Retreiving results for index 71: Carnarvon.
Missing field/result...skipping
-------------------
Retreiving results for index 81: Crotone.
Missing field/result...skipping
-------------------
Retreiving results for index 90: Phalaborwa.
Missing field/result...skipping
-------------------
Retreiving results for index 150: Causapscal.
Missing field/result...skipping
-------------------
Retreiving results for index 197: Rieti.
Missing field/result...skipping
-------------------
Retreiving results for index 296: Jatai.
Missing field/result...skipping
-------------------
Retreiving results for index 300: Bell Ville.
Missing field/result...skipping
-------------------
Retreiving results for index 313: Zatoka.
Missing field/result...skipping
-------------------
Retreiving results for index 322: Caravelas.
Missing field/result...skipping
-------------------
Retreiving results for index 341: Hazorasp.
Mi

In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=[f'{i}' for i in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))